# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pylab as pl
from sklearn import linear_model
%matplotlib inline

In [2]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print df.head()

   admit  gre   gpa  prestige
0      0  380  3.61         3
1      1  660  3.67         3
2      1  800  4.00         1
3      1  640  3.19         4
4      0  520  2.93         4


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [3]:
# frequency table for prestige and whether or not someone was admitted
df.loc[:,('prestige',)] = df.prestige.astype(int)
df.loc[:,('admit',)] = df.admit.astype(int)
df['combined'] = df.apply(lambda row: 'P:{:.0f} A:{}'.format(row.prestige, row.admit), axis = 1)
df.combined.value_counts()

/Users/jakrueger/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/jakrueger/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


P:2 A:0.0    95
P:3 A:0.0    93
P:4 A:0.0    55
P:2 A:1.0    53
P:1 A:1.0    33
P:1 A:0.0    28
P:3 A:1.0    28
P:4 A:1.0    12
Name: combined, dtype: int64

## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [4]:
# convert to int, because float creates annoying column names
dummy_ranks = df.join(pd.get_dummies(df.prestige, prefix='P'))
dummy_ranks

,admit,gre,gpa,prestige,combined,P_1,P_2,P_3,P_4
0,0,380,3.61,3,P:3 A:0.0,0,0,1,0
1,1,660,3.67,3,P:3 A:1.0,0,0,1,0
2,1,800,4.00,1,P:1 A:1.0,1,0,0,0
3,1,640,3.19,4,P:4 A:1.0,0,0,0,1
4,0,520,2.93,4,P:4 A:0.0,0,0,0,1
5,1,760,3.00,2,P:2 A:1.0,0,1,0,0
6,1,560,2.98,1,P:1 A:1.0,1,0,0,0
7,0,400,3.08,2,P:2 A:0.0,0,1,0,0
8,1,540,3.39,3,P:3 A:1.0,0,0,1,0
9,0,700,3.92,2,P:2 A:0.0,0,1,0,0


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: One column per unique value per class variable we are studying.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [5]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'P_1':])
handCalc.head()

,admit,gre,gpa,P_1,P_2,P_3,P_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [8]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
for col in ('P_1', 'P_2', 'P_3', 'P_4'):
    success = 1. * handCalc[handCalc[col] == True]['admit'].sum()
    failure = len(handCalc[(handCalc[col] == True) & (handCalc['admit'] == False)])
    print 'Prestige {} odds are {:.2f} ({:.0f}/{:.0f})'.format(col, success / failure, success, failure)

Prestige P_1 odds are 1.18 (33/28)
Prestige P_2 odds are 0.56 (53/95)
Prestige P_3 odds are 0.30 (28/93)
Prestige P_4 odds are 0.22 (12/55)


In [7]:
pd.crosstab(handCalc.P_1, df.admit)

admit,0,1
P_1,,
0,243,93
1,28,33


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

Answer: The odds are 0.22 for being admitted from a P_4 (prestige for a #1 ranked college)

#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [9]:
print 'Excluding P_4 odds are {:.2f} to 1'.format(1 / 0.22)

Excluding P_4 odds are 4.55 to 1


Answer: If we assume #1 ranked college and "Prestige 4" are equivalent sets, then we can say the odds are approx. 4.5

#### Question 3.3. Calculate the odds ratio.

In [11]:
success = handCalc.admit.sum()
failure = len(handCalc)
odds_ratio = float(success) / failure
print 'Success: {}, Failure: {}, Odds: {}'.format(success, failure, odds_ratio)

Success: 126, Failure: 397, Odds: 0.317380352645


#### Question 3.4. Write this finding in a sentenance:

    Answer: The odds are a little less than 1 out of 3 for admittence

#### Question 3.5. Print the cross tab for prestige_4.

In [12]:
pd.crosstab(handCalc.P_4, df.admit)

admit,0,1
P_4,,
0,216,114
1,55,12


#### Question 3.6. Calculate the OR.

In [13]:
print 12.0 / 55

0.218181818182


#### Question 3.7. Write this finding in a sentence.

Answer: The odds of being admitted from a school with prestige of 4 is 12/55 or roughly 0.22

## Part 4. Analysis

In [14]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'P_2':])
print data.head()

   admit  gre   gpa  P_2  P_3  P_4
0      0  380  3.61    0    1    0
1      1  660  3.67    0    1    0
2      1  800  4.00    0    0    0
3      1  640  3.19    0    0    1
4      0  520  2.93    0    0    1


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [15]:
# manually add the intercept
data['intercept'] = 1.0

#### Question 4.1. Set the covariates to a variable called train_cols.

In [16]:
train_cols = ['gre', 'gpa', 'P_2', 'P_3', 'P_4', 'intercept']

In [49]:
def X_y(df):
    X = df[train_cols]
    y = df['admit']
    return X, y

def y_X(df):
    X = df[train_cols]
    y = df['admit']
    return y, X

#### Question 4.2. Fit the model.

In [50]:
model = linear_model.LogisticRegression()
result = model.fit(*X_y(data))

print 'intercept    =', model.intercept_
print 'coefficients =', model.coef_

intercept    = [-1.09518905]
coefficients = [[ 0.00188666  0.32452811 -0.6200671  -1.16988125 -1.3847737  -1.09518905]]


#### Question 4.3. Print the summary results.

In [19]:
print 'intercept    =', model.intercept_
print 'coefficients =', model.coef_
print 'score = ', model.score(*X_y(data))
model.coef_

intercept    = [-1.09518905]
coefficients = [[ 0.00188666  0.32452811 -0.6200671  -1.16988125 -1.3847737  -1.09518905]]
score =  0.710327455919


array([[ 0.00188666,  0.32452811, -0.6200671 , -1.16988125, -1.3847737 ,
        -1.09518905]])

In [65]:
sm_result = sm.OLS(*y_X(data)).fit()
sm_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  admit   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     8.594
Date:                Wed, 30 Mar 2016   Prob (F-statistic):           9.71e-08
Time:                        11:37:12   Log-Likelihood:                -239.02
No. Observations:                 397   AIC:                             490.0
Df Residuals:                     391   BIC:                             513.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0004      0.000      1.997      0.047      6.48e-06     0.001
gpa            0.1508      0.064      2.349      0.019         0.025     0.277
P_2           -0.1635      0.068     -2.407      0.017        -0.297    -0.030
P_3           -0.2910      0.070     -4.139      0.000        -0.429    -0.153
P_4           -0.3240      0.079     -4.082      0.000        -0.480    -0.168
intercept     -0.2377      0.217     -1.095      0.274        -0.665     0.189
==============================================================================
Omnibus:                      152.312   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.314
Skew:                           0.678   Prob(JB):                     1.19e-11
Kurtosis:                       1.904   Cond. No.                     6.07e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.07e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [67]:
for coeff in model.coef_[0]:
    print 'Coeff: {}, Odd Ratio: {}'.format(coeff, np.exp(coeff))

Coeff: 0.00188665953336, Odd Ratio: 1.0018884404
Coeff: 0.324528109614, Odd Ratio: 1.3833776893
Coeff: -0.620067098559, Odd Ratio: 0.537908343509
Coeff: -1.1698812499, Odd Ratio: 0.310403799559
Coeff: -1.38477370134, Odd Ratio: 0.250380454142
Coeff: -1.09518905282, Odd Ratio: 0.334476367272


In [68]:
sm_model.conf_int()

,0,1
gre,0.000006,0.000837
gpa,0.024562,0.277086
P_2,-0.297107,-0.029977
P_3,-0.429169,-0.152753
P_4,-0.480056,-0.167932
intercept,-0.664512,0.189012


#### Question 4.5. Interpret the OR of Prestige_2.

Answer: 0.54 odds ratio means close to 1 success for almost every 2 failures when the college the applicant comes from has a prestige of 2 if everything else is equal.

#### Question 4.6. Interpret the OR of GPA.

Answer: OR is almost 1.4, which is means the odds increase by nearly 40 percent for each point of GPA.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [69]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [70]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### Question 5.1. Recreate the dummy variables.

In [71]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
# keep only what we need for making predictions
combos = combos.join(pd.get_dummies(combos.prestige, prefix='P'))
del combos['prestige']
del combos['P_1.0']
combos.columns = ['gre', 'gpa', 'intercept', 'P_2', 'P_3', 'P_4']
combos

,gre,gpa,intercept,P_2,P_3,P_4
0,220,2.260000,1,0,0,0
1,220,2.260000,1,1,0,0
2,220,2.260000,1,0,1,0
3,220,2.260000,1,0,0,1
4,220,2.453333,1,0,0,0
5,220,2.453333,1,1,0,0
6,220,2.453333,1,0,1,0
7,220,2.453333,1,0,0,1
8,220,2.646667,1,0,0,0
9,220,2.646667,1,1,0,0


#### Question 5.2. Make predictions on the enumerated dataset.

In [73]:
combos['predict'] = sm_result.predict(combos[train_cols])

In [75]:
combos.tail()

,gre,gpa,intercept,P_2,P_3,P_4,predict
395,800,3.806667,1,0,0,1,0.349652
396,800,4.000000,1,0,0,0,0.702806
397,800,4.000000,1,1,0,0,0.539264
398,800,4.000000,1,0,1,0,0.411845
399,800,4.000000,1,0,0,1,0.378812


#### Question 5.3. Interpret findings for the last 4 observations.

Answer: The 3rd & 4th to last has a 54% & 70% probability (0.70 & 0.54) of admittance, are others below 50%

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.